In [1]:
import altair as alt
import numpy as np
import pandas as pd

import json

In [149]:
blue_mrtk = "#122973"
kpis_columns = ['shiftIdentifier', 'locationId', 'validFrom', 'validTo', 'oee','oeeSetpoint', 
                'availability', 'efficiency', 'quality', 'fpy','fpySetPoint', 'partCount',
                'partCountSetPoint', 'countNIO', 'productivity', 'station']

cycle = pd.read_json("./simulation/CycleTimes.json")
kpi =   pd.read_json("./simulation/KPI.json")


with open(f'DadosSimulados/KPICurrentShift260.json') as f:
    kpi260 = json.load(f)

kpi260["station"]=260

with open(f'DadosSimulados/TargetsExtraSim.json') as f:
    targets = json.load(f)
print(kpi.columns)
def getMetric(t,iskpi=False):
    values260 = [kpi260[d] for d in ['oee', 'fpy', 'partCount', 'countNIO', 'productivity']]
    sizes = {'oee': 3000, 'fpy': 3000, 'partCount':2200, 'countNIO':500, 'productivity':500}
    if (iskpi):
        scaleCor = alt.Scale(domain=[kpi260[d] for d in ['oee', 'fpy', 'partCount', 'countNIO', 'productivity']],range=['red', 'red', 'yellow','green', 'green'])
        scaleSize = alt.Scale(domain=[kpi260[d] for d in ['oee', 'fpy', 'partCount', 'countNIO', 'productivity']],range=[3000, 3000, 2200,5,5])
    else:
        scaleCor = alt.Scale(domain=[0,targets[t]*.95,targets[t]*1.1],range=["red", "yellow", "green"])
        scaleSize=alt.Scale(domain=[0, targets[t]], range=[3000,500] if iskpi else [500, 3000])
    
    display_title = "partCountNOK" if t == "countNIO" else t
    
    if (t in cycle.columns):
        v = cycle[t].iloc[0]
    else:
        v = kpi260[t]
        
    return (alt.Chart().mark_text(fill="white").encode(
        text=alt.value([f"{display_title}: {v}",f"Target: {targets[t]}"]),
        size=alt.value(16)
    )) | (alt.Chart().mark_circle(fill="red").encode(
        size=alt.Size(f"{t}:N" if iskpi else f"{t}:Q", scale=scaleSize) if not iskpi else alt.value(sizes[t]),
        fill=alt.Fill(f"{t}:N" if iskpi else f"{t}:Q",scale=scaleCor)
    ) + alt.Chart().mark_text(fill="white").encode(
        text=alt.Text(f"mean({t}):Q", format='.4'),
        opacity=alt.value(0)
    ))

def getOverview(t, title=""):
    return alt.Chart().mark_text(fill="white").encode(
        text=alt.value(f"{title}"),
        size=alt.value(16)
    ) & (alt.Chart().mark_circle(fill="red").encode(
        size=alt.Size(f"{t}:Q", scale=alt.Scale(range=[80,800])),
        text=alt.value(f"{title}"),
    ) + alt.Chart().mark_text(fill="white").encode(
        text=alt.Text(f"mean({t}):Q", format='.4'),
        opacity=alt.value(0)
    ))
    
def getConfigs(chart):
    return chart.configure(
        background=blue_mrtk
    ).transform_lookup(lookup="station",
        from_=alt.LookupData(data=pd.DataFrame([kpi260]).iloc[[0]], key='station', fields=list(kpis_columns))
    ).configure_axis(
        titleColor="white",
        labelColor="white",
        labelAngle=-45
    ).configure_header(
        titleColor=blue_mrtk,
        titleFontSize=14,
        labelColor=blue_mrtk,
        labelFontSize=14
    ).configure_legend(
        strokeColor='white',
        labelColor='white',
        titleColor='white',
        padding=1,
        tickCount=2,
        disable=True
    ).resolve_scale(
        fill='independent',
        size='independent'
    ).configure_view(
        strokeWidth=0,
        width=100,
        height=100
    ).transform_filter(
        "datum.station == 260"
    )

getConfigs(alt.hconcat(
    (getMetric("totalCycleTime") & getMetric("processTime")), (getMetric("exitTime") & getMetric("changeTime")),  
    data=cycle.iloc[[0]]
))


Index(['shiftIdentifier', 'locationId', 'validFrom', 'validTo', 'oee',
       'oeeSetpoint', 'availability', 'efficiency', 'quality', 'fpy',
       'fpySetPoint', 'partCount', 'partCountSetPoint', 'countNIO',
       'productivity', 'station', 'index'],
      dtype='object')


alt.HConcatChart(...)

In [147]:
getConfigs(alt.hconcat(
    (getMetric("oee", True) & getMetric("fpy",True)), (getMetric("partCount",True) & getMetric("countNIO",True) & getMetric("productivity",True)),  
    data=cycle.iloc[[0]]
))


alt.HConcatChart(...)

In [148]:
getConfigs(alt.hconcat(
    getOverview("totalCycleTime", "Cycle Times"), getOverview("partCount", "Bottleneck"), getOverview("changeTime", "KPIs"),
    data=cycle.iloc[[0]]
))

alt.HConcatChart(...)

In [11]:
blue_mrtk = "#122973"
rule_size = 2

url = "https://raw.githubusercontent.com/tiagodavi70/dashboard_linha_montagem/main/simulation/CycleTimes.json"
# url = "https://raw.githubusercontent.com/tiagodavi70/dashboard_linha_montagem/main/simulation/KPI.json"
attr = "totalCycleTime"

line = alt.Chart(url).mark_line(color="gold").encode(
    y=f"{attr}:Q",
    x=alt.X('index:Q', title="samples", sort="descending")
).transform_filter(
    "datum.station == '260'"
)

rule_linechart = alt.Chart(url).mark_rule(color='green').encode(
    y=alt.Y("target:Q", title=f"seconds"),
    size=alt.value(rule_size)
).transform_filter(
    "datum.station == '260'"
)

(line + rule_linechart).properties(width=600).configure(
    background=blue_mrtk
).configure_axis(
    titleColor="white",
    labelColor="white"
).configure_header(
    titleColor=blue_mrtk,
    titleFontSize=14,
    labelColor=blue_mrtk,
    labelFontSize=14
).transform_filter(
    "datum.index >= 0 && datum.index < 100"
).transform_calculate(
    target="" + str(targets[attr])
)


alt.LayerChart(...)